# Neural Network Performance Visualization

This notebook creates publication-quality visualizations for neural network testing results.

## Features:
- Load testing results from the neural testing module
- Create comprehensive comparison charts (CPLEX vs Randomized vs GCN)
- Generate performance analysis plots
- Export publication-ready figures
- Modern styling with error bars and statistical significance

**Inspired by NeuralTesting.py but with clean, modular design using TestingNeuralNetwork.py**

## Setup and Imports

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Tuple, Optional
import pandas as pd

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Add the current directory to Python path for imports
current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.append(str(current_dir))
if str(current_dir.parent) not in sys.path:
    sys.path.append(str(current_dir.parent))

# Import our modules
from commons import open_file
from Testing.TestingNeuralNetwork import test_multiple_graphs, analyze_results

print("All modules imported successfully!")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")

ModuleNotFoundError: No module named 'seaborn'

## Configuration and Data Loading

In [ ]:
# Visualization configuration
VIZ_CONFIG = {
    'results_directory': './neural_testing_results',
    'output_directory': './publication_figures',
    'figure_format': 'png',
    'dpi': 300,
    'figure_size': (12, 8),
    'font_size': 12,
    'title_size': 14,
    'legend_size': 10
}

# Create output directory
os.makedirs(VIZ_CONFIG['output_directory'], exist_ok=True)

# Set matplotlib parameters
plt.rcParams.update({
    'font.size': VIZ_CONFIG['font_size'],
    'axes.titlesize': VIZ_CONFIG['title_size'],
    'axes.labelsize': VIZ_CONFIG['font_size'],
    'xtick.labelsize': VIZ_CONFIG['font_size'],
    'ytick.labelsize': VIZ_CONFIG['font_size'],
    'legend.fontsize': VIZ_CONFIG['legend_size'],
    'figure.titlesize': VIZ_CONFIG['title_size']
})

print("Configuration loaded:")
for key, value in VIZ_CONFIG.items():
    print(f"  {key}: {value}")

## Load Test Results

In [ ]:
# Load neural network test results
results_file = os.path.join(VIZ_CONFIG['results_directory'], 'neural_network_test_results.pkl')

try:
    detailed_results = open_file(results_file)
    
    # Extract components
    test_results = detailed_results['individual_results']
    results_by_size = detailed_results['results_by_size']
    analysis = detailed_results['analysis']
    testing_config = detailed_results['testing_config']
    
    print(f"✓ Loaded test results successfully")
    print(f"  Total tests: {len(test_results)}")
    print(f"  Graph sizes: {sorted(results_by_size.keys())}")
    print(f"  Analysis keys: {list(analysis.keys())}")
    
except FileNotFoundError:
    print(f"✗ Results file not found: {results_file}")
    print("Please run the neural_network_testing.ipynb notebook first to generate results.")
    raise
except Exception as e:
    print(f"✗ Error loading results: {e}")
    raise

## Sample Data for Algorithm Comparison

For demonstration purposes, we'll create sample CPLEX and randomized algorithm results.
In practice, you would load these from your actual experimental data.

In [ ]:
# Create sample data for comparison (replace with actual CPLEX/randomized results)
def generate_sample_baseline_data(results_by_size: Dict, noise_factor: float = 0.1) -> Dict:
    """
    Generate sample baseline data for CPLEX and randomized algorithms.
    In practice, replace this with actual experimental results.
    """
    baseline_data = {}
    
    for size, data in results_by_size.items():
        if not data['simple']['cut_values']:
            continue
            
        gcn_cuts = data['post_processed']['cut_values']
        
        # Generate realistic CPLEX results (typically better than GCN)
        cplex_cuts = []
        for cut in gcn_cuts:
            # CPLEX typically achieves 10-20% better results than GCN
            cplex_cut = cut * (1.1 + np.random.normal(0, noise_factor))
            cplex_cuts.append(max(cut, cplex_cut))  # Ensure CPLEX is at least as good
        
        # Generate realistic randomized results (typically worse than GCN)
        randomized_cuts = []
        for cut in gcn_cuts:
            # Randomized typically achieves 5-15% worse results than GCN
            randomized_cut = cut * (0.9 + np.random.normal(0, noise_factor))
            randomized_cuts.append(randomized_cut)
        
        # Generate runtime data
        gcn_times = data['post_processed']['times']
        cplex_times = [t * (10 + np.random.exponential(5)) for t in gcn_times]  # CPLEX much slower
        randomized_times = [t * (0.1 + np.random.normal(0, 0.05)) for t in gcn_times]  # Randomized much faster
        
        baseline_data[size] = {
            'cplex': {
                'cut_values': cplex_cuts,
                'times': cplex_times,
                'std_percent': [np.random.uniform(2, 5) for _ in cplex_cuts]  # Small variance for CPLEX
            },
            'randomized': {
                'cut_values': randomized_cuts,
                'times': randomized_times,
                'std_percent': [np.random.uniform(8, 15) for _ in randomized_cuts]  # Higher variance
            },
            'gcn_simple': {
                'cut_values': data['simple']['cut_values'],
                'times': data['simple']['times'],
                'std_percent': [np.random.uniform(3, 8) for _ in data['simple']['cut_values']]
            },
            'gcn_post': {
                'cut_values': data['post_processed']['cut_values'],
                'times': data['post_processed']['times'],
                'std_percent': [np.random.uniform(3, 8) for _ in data['post_processed']['cut_values']]
            }
        }
    
    return baseline_data

# Generate sample data
baseline_data = generate_sample_baseline_data(results_by_size)

print("Sample baseline data generated for comparison:")
for size in sorted(baseline_data.keys()):
    data = baseline_data[size]
    print(f"  Size {size}: CPLEX={np.mean(data['cplex']['cut_values']):.1f}, "
          f"Randomized={np.mean(data['randomized']['cut_values']):.1f}, "
          f"GCN={np.mean(data['gcn_post']['cut_values']):.1f}")

## Advanced Visualization Functions

In [ ]:
def create_algorithm_comparison_chart(baseline_data: Dict, title: str = 'Algorithm Performance Comparison',
                                     y_label: str = 'Cut Value', show_percentages: bool = True,
                                     save_path: Optional[str] = None) -> None:
    """
    Create a comprehensive algorithm comparison chart similar to NeuralTesting.py barPlot_3_dot.
    """
    sizes = sorted(baseline_data.keys())
    n_groups = len(sizes)
    x = np.arange(n_groups)
    bar_width = 0.2
    
    fig, ax = plt.subplots(figsize=VIZ_CONFIG['figure_size'])
    
    # Colors for each algorithm
    colors = {
        'cplex': '#87CEEB',      # Sky blue
        'randomized': '#FFA500',  # Orange
        'gcn_simple': '#FF6347',  # Red
        'gcn_post': '#90EE90'     # Light green
    }
    
    labels = {
        'cplex': 'CPLEX',
        'randomized': 'Randomized Algorithm',
        'gcn_simple': 'GCN',
        'gcn_post': 'GCN with Post-processing'
    }
    
    algorithms = ['cplex', 'randomized', 'gcn_simple', 'gcn_post']
    
    # Plot bars for each algorithm
    bars = {}
    for i, alg in enumerate(algorithms):
        means = [np.mean(baseline_data[size][alg]['cut_values']) for size in sizes]
        stds = [np.std(baseline_data[size][alg]['cut_values']) for size in sizes]
        
        bars[alg] = ax.bar(x + i * bar_width, means, bar_width, 
                          label=labels[alg], color=colors[alg], alpha=0.8,
                          yerr=stds, capsize=5, error_kw={'color': 'black', 'alpha': 0.7})
    
    # Add percentage annotations if requested
    if show_percentages:
        for i, size in enumerate(sizes):
            cplex_mean = np.mean(baseline_data[size]['cplex']['cut_values'])
            
            for j, alg in enumerate(algorithms[1:], 1):  # Skip CPLEX (baseline)
                alg_mean = np.mean(baseline_data[size][alg]['cut_values'])
                percentage = (alg_mean / cplex_mean * 100) if cplex_mean > 0 else 0
                
                x_pos = x[i] + j * bar_width
                y_pos = alg_mean * 0.5
                
                ax.text(x_pos, y_pos, f'{percentage:.0f}%',
                       ha='center', va='center', fontweight='bold',
                       color='white' if percentage < 90 else 'black',
                       fontsize=VIZ_CONFIG['font_size'] - 2)
    
    # Formatting
    ax.set_xlabel('Graph Size (nodes)')
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.set_xticks(x + bar_width * 1.5)
    ax.set_xticklabels(sizes)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=VIZ_CONFIG['dpi'], bbox_inches='tight')
        print(f"Chart saved to: {save_path}")
    
    plt.show()

def create_runtime_comparison_chart(baseline_data: Dict, title: str = 'Runtime Comparison',
                                  save_path: Optional[str] = None) -> None:
    """
    Create a runtime comparison chart with time annotations.
    """
    sizes = sorted(baseline_data.keys())
    n_groups = len(sizes)
    x = np.arange(n_groups)
    bar_width = 0.25
    
    fig, ax = plt.subplots(figsize=VIZ_CONFIG['figure_size'])
    
    # Colors
    colors = ['#FFA500', '#FF6347', '#90EE90']  # Orange, Red, Green
    algorithms = ['randomized', 'gcn_simple', 'gcn_post']
    labels = ['Randomized Algorithm', 'GCN', 'GCN with Post-processing']
    
    # Plot bars
    for i, alg in enumerate(algorithms):
        means = [np.mean(baseline_data[size][alg]['times']) for size in sizes]
        stds = [np.std(baseline_data[size][alg]['times']) for size in sizes]
        
        bars = ax.bar(x + i * bar_width, means, bar_width, 
                     label=labels[i], color=colors[i], alpha=0.8,
                     yerr=stds, capsize=5)
        
        # Add time annotations on top of bars
        for j, (mean, std) in enumerate(zip(means, stds)):
            x_pos = x[j] + i * bar_width
            y_pos = mean + std + mean * 0.05  # Slightly above error bar
            
            ax.text(x_pos, y_pos, f'{mean:.2f}s',
                   ha='center', va='bottom', fontweight='bold',
                   fontsize=VIZ_CONFIG['font_size'] - 2)
    
    # Formatting
    ax.set_xlabel('Graph Size (nodes)')
    ax.set_ylabel('Runtime (seconds)')
    ax.set_title(title)
    ax.set_xticks(x + bar_width)
    ax.set_xticklabels(sizes)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=VIZ_CONFIG['dpi'], bbox_inches='tight')
        print(f"Runtime chart saved to: {save_path}")
    
    plt.show()

def create_scalability_line_plot(baseline_data: Dict, title: str = 'Algorithm Scalability Analysis',
                                save_path: Optional[str] = None) -> None:
    """
    Create a line plot showing how algorithms scale with graph size.
    """
    sizes = sorted(baseline_data.keys())
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    algorithms = ['cplex', 'randomized', 'gcn_simple', 'gcn_post']
    colors = ['#87CEEB', '#FFA500', '#FF6347', '#90EE90']
    labels = ['CPLEX', 'Randomized', 'GCN', 'GCN + Post-proc']
    markers = ['o', 's', '^', 'D']
    
    # Performance scalability (left plot)
    for i, alg in enumerate(algorithms):
        means = [np.mean(baseline_data[size][alg]['cut_values']) for size in sizes]
        stds = [np.std(baseline_data[size][alg]['cut_values']) for size in sizes]
        
        ax1.errorbar(sizes, means, yerr=stds, label=labels[i], 
                    color=colors[i], marker=markers[i], linewidth=2, 
                    markersize=8, capsize=5)
    
    ax1.set_xlabel('Graph Size (nodes)')
    ax1.set_ylabel('Average Cut Value')
    ax1.set_title('Performance Scalability')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Runtime scalability (right plot, excluding CPLEX for better scale)
    for i, alg in enumerate(algorithms[1:], 1):  # Skip CPLEX
        means = [np.mean(baseline_data[size][alg]['times']) for size in sizes]
        stds = [np.std(baseline_data[size][alg]['times']) for size in sizes]
        
        ax2.errorbar(sizes, means, yerr=stds, label=labels[i], 
                    color=colors[i], marker=markers[i], linewidth=2, 
                    markersize=8, capsize=5)
    
    ax2.set_xlabel('Graph Size (nodes)')
    ax2.set_ylabel('Average Runtime (seconds)')
    ax2.set_title('Runtime Scalability')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=VIZ_CONFIG['title_size'] + 2)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=VIZ_CONFIG['dpi'], bbox_inches='tight')
        print(f"Scalability plot saved to: {save_path}")
    
    plt.show()

def create_improvement_analysis(baseline_data: Dict, title: str = 'GCN Post-processing Effectiveness',
                              save_path: Optional[str] = None) -> None:
    """
    Create a detailed analysis of GCN post-processing improvements.
    """
    sizes = sorted(baseline_data.keys())
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Absolute improvement
    improvements = []
    for size in sizes:
        simple_cuts = baseline_data[size]['gcn_simple']['cut_values']
        post_cuts = baseline_data[size]['gcn_post']['cut_values']
        size_improvements = [post - simple for simple, post in zip(simple_cuts, post_cuts)]
        improvements.extend(size_improvements)
    
    ax1.hist(improvements, bins=20, alpha=0.7, color='green', edgecolor='black')
    ax1.axvline(x=0, color='red', linestyle='--', alpha=0.8, label='No improvement')
    ax1.axvline(x=np.mean(improvements), color='blue', linestyle='-', alpha=0.8, 
                label=f'Mean: {np.mean(improvements):.1f}')
    ax1.set_xlabel('Cut Value Improvement')
    ax1.set_ylabel('Number of Graphs')
    ax1.set_title('Distribution of Post-processing Improvements')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Improvement by graph size
    size_improvements = []
    size_labels = []
    for size in sizes:
        simple_cuts = baseline_data[size]['gcn_simple']['cut_values']
        post_cuts = baseline_data[size]['gcn_post']['cut_values']
        improvements = [(post - simple) / simple * 100 if simple > 0 else 0 
                       for simple, post in zip(simple_cuts, post_cuts)]
        size_improvements.append(improvements)
        size_labels.append(f'{size} nodes')
    
    ax2.boxplot(size_improvements, labels=size_labels)
    ax2.axhline(y=0, color='red', linestyle='--', alpha=0.8)
    ax2.set_ylabel('Improvement (%)')
    ax2.set_title('Improvement Distribution by Graph Size')
    ax2.grid(True, alpha=0.3)
    
    # 3. Success rate by size
    success_rates = []
    for size in sizes:
        simple_cuts = baseline_data[size]['gcn_simple']['cut_values']
        post_cuts = baseline_data[size]['gcn_post']['cut_values']
        successes = sum(1 for simple, post in zip(simple_cuts, post_cuts) if post > simple)
        rate = successes / len(simple_cuts) * 100 if simple_cuts else 0
        success_rates.append(rate)
    
    ax3.bar(range(len(sizes)), success_rates, color='lightblue', alpha=0.8)
    ax3.set_xlabel('Graph Size')
    ax3.set_ylabel('Success Rate (%)')
    ax3.set_title('Post-processing Success Rate by Graph Size')
    ax3.set_xticks(range(len(sizes)))
    ax3.set_xticklabels([f'{size}' for size in sizes])
    ax3.set_ylim(0, 100)
    ax3.grid(True, alpha=0.3)
    
    # 4. Runtime overhead
    runtime_ratios = []
    for size in sizes:
        simple_times = baseline_data[size]['gcn_simple']['times']
        post_times = baseline_data[size]['gcn_post']['times']
        ratios = [post / simple if simple > 0 else 1 for simple, post in zip(simple_times, post_times)]
        runtime_ratios.append(np.mean(ratios))
    
    ax4.plot(sizes, runtime_ratios, 'ro-', linewidth=2, markersize=8)
    ax4.axhline(y=1, color='black', linestyle='--', alpha=0.8, label='No overhead')
    ax4.set_xlabel('Graph Size (nodes)')
    ax4.set_ylabel('Runtime Ratio (Post/Simple)')
    ax4.set_title('Post-processing Runtime Overhead')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=VIZ_CONFIG['title_size'] + 2)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=VIZ_CONFIG['dpi'], bbox_inches='tight')
        print(f"Improvement analysis saved to: {save_path}")
    
    plt.show()

print("Advanced visualization functions defined successfully!")

## Algorithm Performance Comparison

Create comprehensive comparison charts between different algorithms

In [ ]:
# Create algorithm comparison chart for cut values
save_path = os.path.join(VIZ_CONFIG['output_directory'], f'algorithm_comparison_cut_values.{VIZ_CONFIG["figure_format"]}')
create_algorithm_comparison_chart(
    baseline_data=baseline_data,
    title='3-Way Max-Cut: Algorithm Performance Comparison',
    y_label='Maximum Cut Value',
    show_percentages=True,
    save_path=save_path
)

## Runtime Analysis

Compare algorithm runtime performance

In [ ]:
# Create runtime comparison chart
save_path = os.path.join(VIZ_CONFIG['output_directory'], f'runtime_comparison.{VIZ_CONFIG["figure_format"]}')
create_runtime_comparison_chart(
    baseline_data=baseline_data,
    title='Algorithm Runtime Comparison',
    save_path=save_path
)

## Scalability Analysis

Examine how algorithms scale with increasing graph size

In [ ]:
# Create scalability analysis
save_path = os.path.join(VIZ_CONFIG['output_directory'], f'scalability_analysis.{VIZ_CONFIG["figure_format"]}')
create_scalability_line_plot(
    baseline_data=baseline_data,
    title='Algorithm Scalability: Performance vs Graph Size',
    save_path=save_path
)

## Post-processing Effectiveness Analysis

Detailed analysis of GCN post-processing improvements

In [ ]:
# Create post-processing analysis
save_path = os.path.join(VIZ_CONFIG['output_directory'], f'post_processing_analysis.{VIZ_CONFIG["figure_format"]}')
create_improvement_analysis(
    baseline_data=baseline_data,
    title='GCN Post-processing: Comprehensive Effectiveness Analysis',
    save_path=save_path
)

## Custom Analysis: Two-Algorithm Comparison

Simplified comparison between GCN variants (similar to NeuralTesting.py barPlot_2)

In [ ]:
def create_gcn_comparison_chart(baseline_data: Dict, title: str = 'GCN Variants Comparison',
                               save_path: Optional[str] = None) -> None:
    """
    Create a focused comparison between GCN variants.
    """
    sizes = sorted(baseline_data.keys())
    n_groups = len(sizes)
    x = np.arange(n_groups)
    bar_width = 0.35
    
    fig, ax = plt.subplots(figsize=VIZ_CONFIG['figure_size'])
    
    # Data for GCN variants
    simple_means = [np.mean(baseline_data[size]['gcn_simple']['cut_values']) for size in sizes]
    simple_stds = [np.std(baseline_data[size]['gcn_simple']['cut_values']) for size in sizes]
    
    post_means = [np.mean(baseline_data[size]['gcn_post']['cut_values']) for size in sizes]
    post_stds = [np.std(baseline_data[size]['gcn_post']['cut_values']) for size in sizes]
    
    # Create bars
    bars1 = ax.bar(x - bar_width/2, simple_means, bar_width, 
                   label='GCN', color='#FF6347', alpha=0.8,
                   yerr=simple_stds, capsize=5)
    
    bars2 = ax.bar(x + bar_width/2, post_means, bar_width, 
                   label='GCN with Post-processing', color='#90EE90', alpha=0.8,
                   yerr=post_stds, capsize=5)
    
    # Add value annotations on top of bars
    for i, (simple_mean, post_mean, simple_std, post_std) in enumerate(zip(simple_means, post_means, simple_stds, post_stds)):
        # Simple GCN annotation
        ax.text(x[i] - bar_width/2, simple_mean + simple_std + simple_mean * 0.02,
                f'{simple_mean:.0f}', ha='center', va='bottom', fontweight='bold')
        
        # Post-processed GCN annotation
        ax.text(x[i] + bar_width/2, post_mean + post_std + post_mean * 0.02,
                f'{post_mean:.0f}', ha='center', va='bottom', fontweight='bold')
        
        # Improvement percentage in the middle
        improvement_pct = (post_mean - simple_mean) / simple_mean * 100 if simple_mean > 0 else 0
        max_height = max(simple_mean + simple_std, post_mean + post_std)
        ax.text(x[i], max_height + max_height * 0.05,
                f'{improvement_pct:+.1f}%', ha='center', va='bottom', 
                fontweight='bold', color='blue', fontsize=VIZ_CONFIG['font_size'] - 1)
    
    # Formatting
    ax.set_xlabel('Graph Size (nodes)')
    ax.set_ylabel('Maximum Cut Value')
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(sizes)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=VIZ_CONFIG['dpi'], bbox_inches='tight')
        print(f"GCN comparison chart saved to: {save_path}")
    
    plt.show()

# Create GCN variants comparison
save_path = os.path.join(VIZ_CONFIG['output_directory'], f'gcn_variants_comparison.{VIZ_CONFIG["figure_format"]}')
create_gcn_comparison_chart(
    baseline_data=baseline_data,
    title='GCN Performance: With vs Without Post-processing',
    save_path=save_path
)

## Statistical Significance Testing

In [ ]:
from scipy import stats

def perform_statistical_analysis(baseline_data: Dict) -> None:
    """
    Perform statistical significance testing on the results.
    """
    print("Statistical Significance Analysis")
    print("=" * 50)
    
    for size in sorted(baseline_data.keys()):
        data = baseline_data[size]
        
        simple_cuts = data['gcn_simple']['cut_values']
        post_cuts = data['gcn_post']['cut_values']
        
        # Paired t-test (since we're comparing the same graphs)
        t_stat, p_value = stats.ttest_rel(post_cuts, simple_cuts)
        
        # Effect size (Cohen's d)
        differences = [post - simple for post, simple in zip(post_cuts, simple_cuts)]
        cohens_d = np.mean(differences) / np.std(differences) if np.std(differences) > 0 else 0
        
        # Determine significance
        significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
        
        print(f"\nGraph Size {size} nodes:")
        print(f"  Mean improvement: {np.mean(differences):.2f} ± {np.std(differences):.2f}")
        print(f"  t-statistic: {t_stat:.3f}")
        print(f"  p-value: {p_value:.6f} {significance}")
        print(f"  Cohen's d: {cohens_d:.3f}")
        print(f"  Effect size: {'Large' if abs(cohens_d) > 0.8 else 'Medium' if abs(cohens_d) > 0.5 else 'Small'}")
    
    print(f"\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")

# Perform statistical analysis
perform_statistical_analysis(baseline_data)

## Summary Report Generation

In [ ]:
def generate_visualization_summary(baseline_data: Dict, analysis: Dict) -> str:
    """
    Generate a comprehensive summary of all visualizations and findings.
    """
    sizes = sorted(baseline_data.keys())
    total_graphs = sum(len(baseline_data[size]['gcn_simple']['cut_values']) for size in sizes)
    
    # Calculate overall statistics
    all_improvements = []
    all_success_rates = []
    
    for size in sizes:
        simple_cuts = baseline_data[size]['gcn_simple']['cut_values']
        post_cuts = baseline_data[size]['gcn_post']['cut_values']
        
        improvements = [post - simple for simple, post in zip(simple_cuts, post_cuts)]
        all_improvements.extend(improvements)
        
        successes = sum(1 for imp in improvements if imp > 0)
        success_rate = successes / len(improvements) * 100 if improvements else 0
        all_success_rates.append(success_rate)
    
    avg_improvement = np.mean(all_improvements)
    overall_success_rate = np.mean(all_success_rates)
    
    summary = f"""Neural Network Visualization Summary Report
{'='*60}

Dataset Overview:
  Total test graphs: {total_graphs}
  Graph sizes tested: {sizes}
  Graphs per size: {[len(baseline_data[size]['gcn_simple']['cut_values']) for size in sizes]}

Post-processing Performance:
  Average improvement: {avg_improvement:+.2f} cut value
  Overall success rate: {overall_success_rate:.1f}%
  Standard deviation: {np.std(all_improvements):.2f}

Algorithm Rankings (by average performance):"""
    
    # Calculate average performance for each algorithm
    alg_performance = {}
    for alg in ['cplex', 'gcn_post', 'gcn_simple', 'randomized']:
        all_cuts = []
        for size in sizes:
            all_cuts.extend(baseline_data[size][alg]['cut_values'])
        alg_performance[alg] = np.mean(all_cuts)
    
    # Sort by performance
    sorted_algs = sorted(alg_performance.items(), key=lambda x: x[1], reverse=True)
    
    alg_names = {
        'cplex': 'CPLEX (Integer Solver)',
        'gcn_post': 'GCN with Post-processing',
        'gcn_simple': 'GCN (Simple)',
        'randomized': 'Randomized Algorithm'
    }
    
    for i, (alg, avg_cut) in enumerate(sorted_algs, 1):
        summary += f"\n  {i}. {alg_names[alg]}: {avg_cut:.1f}"
    
    summary += f"""

Generated Visualizations:
  1. Algorithm Performance Comparison (Cut Values)
  2. Runtime Comparison Analysis
  3. Scalability Analysis (Performance vs Graph Size)
  4. Post-processing Effectiveness Analysis
  5. GCN Variants Comparison
  6. Statistical Significance Testing

Key Findings:
  - Post-processing {'improves' if avg_improvement > 0 else 'decreases'} GCN performance on average
  - Success rate varies from {min(all_success_rates):.1f}% to {max(all_success_rates):.1f}% across graph sizes
  - {'CPLEX' if sorted_algs[0][0] == 'cplex' else 'GCN'} achieves the best overall performance
  - Randomized algorithm provides fastest runtime but lowest quality

Figures saved to: {VIZ_CONFIG['output_directory']}
Format: {VIZ_CONFIG['figure_format'].upper()}, DPI: {VIZ_CONFIG['dpi']}

{'='*60}"""
    
    return summary

# Generate and display summary
summary_report = generate_visualization_summary(baseline_data, analysis)
print(summary_report)

# Save summary to file
summary_file = os.path.join(VIZ_CONFIG['output_directory'], 'visualization_summary.txt')
with open(summary_file, 'w') as f:
    f.write(summary_report)

print(f"\nVisualization summary saved to: {summary_file}")
print(f"All figures saved to: {VIZ_CONFIG['output_directory']}")